# Counties Ranked By Exodus 

Get net amount of people who left B for other counties 

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("county-to-county-movement-california-filtered.csv")
df["net-flow-from-b-to-a"] = df["net-flow-from-b-to-a"].str.replace(',', '').astype(int)

In [3]:
df.head()

,State Code of Geography A,FIPS County Code of Geography A,State/U.S. Island Area/Foreign Region Code of Geography B,FIPS County Code of Geography B,county-a,county-b,flow-from-b-to-a,flow-from-a-to-b,net-flow-from-b-to-a,gross-migration
0,6,1,6,5,Alameda County,Amador County,24,77,-53,101
1,6,1,6,7,Alameda County,Butte County,298,863,-565,"1,161"
2,6,1,6,9,Alameda County,Calaveras County,16,190,-174,206
3,6,1,6,11,Alameda County,Colusa County,0,8,-8,8
4,6,1,6,13,Alameda County,Contra Costa County,"9,550","16,443",-6893,"25,993"


In [11]:
# Aggregate how many people left each county
counties = set(df["county-b"])
new_df = pd.DataFrame(columns=['code', 'county', 'net-exodus'] + ["flow-to-" + county.replace(" ", "-") for county in counties])

for county_b in counties: 
    row = {
        "code": df.loc[df["county-b"] == county_b].iloc[0]["FIPS County Code of Geography B"],
        "county": county_b,
        "net-exodus": df.loc[df["county-b"] == county_b]["net-flow-from-b-to-a"].sum()
    }
    
    for county_a in counties: 
        if county_a != county_b:
            flow = df.loc[(df["county-b"] == county_b) & (df["county-a"] == county_a)]
            if len(flow): row["flow-to-" + county_a.replace(" ", "-")] = flow.iloc[0]["net-flow-from-b-to-a"]
    
    new_df = new_df.append(row, ignore_index=True)

In [12]:
new_df.head()

,code,county,net-exodus,flow-to-San-Bernardino-County,flow-to-San-Diego-County,flow-to-Shasta-County,flow-to-San-Luis-Obispo-County,flow-to-Mariposa-County,flow-to-Contra-Costa-County,flow-to-Santa-Clara-County,...,flow-to-Sutter-County,flow-to-Butte-County,flow-to-Tulare-County,flow-to-Colusa-County,flow-to-Del-Norte-County,flow-to-Humboldt-County,flow-to-Merced-County,flow-to-Ventura-County,flow-to-Amador-County,flow-to-Santa-Cruz-County
0,71,San Bernardino County,-23855,NaN,-2519,8,9,15,30,53,...,-132,-92,517,NaN,0,12,40,-182,-89,-154
1,73,San Diego County,16368,2519,NaN,164,319,14,459,297,...,53,442,4,NaN,-71,357,175,127,28,376
2,89,Shasta County,62,-8,-164,NaN,10,-7,-115,-486,...,13,390,124,29,-398,-82,25,229,11,-149
3,79,San Luis Obispo County,-5386,-9,-319,-10,NaN,NaN,-265,-287,...,74,-104,-15,-24,-45,-47,-10,-85,-84,-298
4,43,Mariposa County,-123,-15,-14,7,NaN,NaN,-134,NaN,...,5,NaN,NaN,NaN,NaN,-5,86,NaN,NaN,-2


In [10]:
new_df = new_df.sort_values(by=['net-exodus'], ascending = False)
new_df.head()

,code,county,net-exodus
38,37,Los Angeles County,70206
2,85,Santa Clara County,18432
56,73,San Diego County,16368
29,75,San Francisco County,13113
18,1,Alameda County,6802


In [8]:
# Export data to csv
new_df.to_csv (r'../../data/california-migration/county-net-exodus.csv', index = None, header=True)